### General purpose data cleaning. 
#### If you have cleaned a file let's keep track of what we did here.

In [20]:
import pandas as pd
import numpy as np
import geocoder
import requests
from geopy.geocoders import Nominatim

from File_Utilities import FileTools
import DB_Utilities
DBTools = DB_Utilities.DBTools()  # instantiate the class
FileTools.MYDIR = "./data/"

### Making crosswalk for: merging on city name and state name to fips codes.

In [2]:
patents = pd.read_csv(FileTools.get_full_file_path('reduced_2009_2019.csv'),low_memory=False )

# list_of_cities that do not have a county listed.
no_county = patents[patents.county.isna()]
list_of_cities = pd.DataFrame(no_county.groupby(['state', 'city']).size()).reset_index().rename(columns={0: 'count'})
FileTools.save_df_as_csv(list_of_cities, 'list_of_cities.csv')


# THIS IS WHERE WE DID SOME GEOCODING IN ARCGIS PRO
# Geocoded on just the city and state name. 


# Bring the new crosswalk back
city_crosswalk = pd.read_csv(FileTools.get_full_file_path('crosswalk_city_geocoded.csv'),low_memory=False )
city_crosswalk['GEOID'] = city_crosswalk['STATEFP'].astype(str).str.replace(".0", "").str.zfill(2)+city_crosswalk['COUNTYFP'].astype(str).str.replace(".0", "").str.zfill(3)
city_crosswalk_final = city_crosswalk[['USER_state', 'USER_city', 'STATEFP', 'COUNTYFP', 'GEOID']].copy()
city_crosswalk_final.rename(columns={"USER_state": "state", "USER_city": "city", "STATEFP": "STATEFP", "COUNTYFP": "COUNTYFP"}, inplace=True)

FileTools.save_df_as_csv(city_crosswalk_final, 'crosswalk_city_state_name_to_city_state_fips.csv')

C:\Users\micha\AppData\Local\Temp\ipykernel_10768\1061403620.py:15: FutureWarning: The default value of regex will change from True to False in a future version.
  city_crosswalk['GEOID'] = city_crosswalk['STATEFP'].astype(str).str.replace(".0", "").str.zfill(2)+city_crosswalk['COUNTYFP'].astype(str).str.replace(".0", "").str.zfill(3)


'crosswalk_city_state_name_to_city_state_fips.csv'

In [41]:
def geocode_city_state(city, state):
    # Initialize Nominatim API
    geolocator = Nominatim(user_agent="UVaPatent22Capstone_DataScience")

    # location = geolocator.geocode(f"{city} , {state}")
    location = geolocator.geocode({'city':city, 'state':state})

    print('location')
    print(location.raw)

    Latitude = str(location.latitude)
    Longitude = str(location.longitude)

    # print("The latitude of the location is: ", Latitude)
    # print("The longitude of the location is: ", Longitude)
    reverse_location = geolocator.reverse(Latitude+","+Longitude)    
    print(reverse_location)

   
    url = f'https://geocoding.geo.census.gov/geocoder/geographies/coordinates?x={Longitude}&y={Latitude}&benchmark=4&vintage=4&format=json'
    print (f'https://geocoding.geo.census.gov/geocoder/geographies/coordinates?x={Longitude}&y={Latitude}&benchmark=4&vintage=4')
    response = requests.get(url)
    data = response.json()


    county_GEOID = data['result']['geographies']['Counties'][0]['GEOID']    
    county_fip = data['result']['geographies']['Counties'][0]['COUNTY']
    county_BASENAME = data['result']['geographies']['Counties'][0]['BASENAME']

    state_fip = data['result']['geographies']['States'][0]['STATE']
    state_BASENAME = data['result']['geographies']['States'][0]['BASENAME']
    state_STUSAB = data['result']['geographies']['States'][0]['STUSAB'] 

    city_BASENAME = data['result']['geographies']['County Subdivisions'][0]['BASENAME'] 
    city_NAME = data['result']['geographies']['County Subdivisions'][0]['NAME']     

    return {
    'original_state' : state,
    'original_city' : city,
    'county_GEOID' : county_GEOID ,
    'county_fip' :county_fip, 
    'county_BASENAME' :county_BASENAME ,
    'state_fip' :state_fip ,
    'state_BASENAME' : state_BASENAME ,
    'state_STUSAB' : state_STUSAB ,
    'city_BASENAME' : city_BASENAME, 
    'city_NAME' : city_NAME ,
    'status' : state_STUSAB == state }
    
    # To see the results in an html form.
    # https://geocoding.geo.census.gov/geocoder/geographies/coordinates?x=-70.1441014&y=44.671539&benchmark=4&vintage=4

geocode_city_state("Farmington", "MA")

location
{'place_id': 298003928, 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright', 'osm_type': 'relation', 'osm_id': 136899, 'boundingbox': ['44.6232928', '44.692138', '-93.2076268', '-93.1163862'], 'lat': '44.6402434', 'lon': '-93.1435497', 'display_name': 'Farmington, Dakota County, Minnesota, 55024, United States', 'class': 'boundary', 'type': 'administrative', 'importance': 0.5748773825523825, 'icon': 'https://nominatim.openstreetmap.org/ui/mapicons/poi_boundary_administrative.p.20.png'}
302, 4th Street, Farmington, Dakota County, Minnesota, 55024, United States
https://geocoding.geo.census.gov/geocoder/geographies/coordinates?x=-93.1435497&y=44.6402434&benchmark=4&vintage=4


{'original_state': 'MA',
 'original_city': 'Farmington',
 'county_GEOID': '27037',
 'county_fip': '037',
 'county_BASENAME': 'Dakota',
 'state_fip': '27',
 'state_BASENAME': 'Minnesota',
 'state_STUSAB': 'MN',
 'city_BASENAME': 'Farmington',
 'city_NAME': 'Farmington city',
 'status': False}

## County Business Patterns 
### Pivoting tables so county is row and all predictors are columns

In [19]:
# Load in the data.
_cbp_lq_3digit_naics_all_years_df = FileTools.load_df_from_parquet('cbp_lq_3digit_naics_all_years.gzip')
_cbp_lq_3digit_naics_all_years_df.columns


Index(['fipstate', 'fipscty', 'naics', 'emp_nf', 'emp_county_naics', 'qp1_nf',
       'qp1', 'ap_nf', 'ap', 'est', 'n<5', 'n5_9', 'n10_19', 'n20_49',
       'n50_99', 'n100_249', 'n250_499', 'n500_999', 'n1000', 'n1000_1',
       'n1000_2', 'n1000_3', 'n1000_4', 'naics_level', 'county_fips',
       'emp_county', 'percent_of_county_emp', 'percent_of_state_emp',
       'emp_state_naics', 'emp_state', 'percent_of_us_emp', 'emp_us_naics',
       'emp_us', 'location_quotient_county_state',
       'location_quotient_county_us', 'year', 'censtate', 'cencty', 'empflag',
       'n1_4'],
      dtype='object')

In [28]:
all_data_cbp = pd.DataFrame()
for i in range(2010,2021): 
# for i in range(2020,2021): 
    year = str(i)
    print(f'Working on year : {year}')    

    # Filter by the year we want.
    temp_cbp_lq_3digit_naics_all_years_df = _cbp_lq_3digit_naics_all_years_df[
        (_cbp_lq_3digit_naics_all_years_df['year'] == year) 
        & (_cbp_lq_3digit_naics_all_years_df['naics_level'] == 3)
        ][[
            'county_fips','qp1', 'ap', 'est','naics','location_quotient_county_state','location_quotient_county_us']].copy()


    # Get some aggregations by county
    group_cbp_lq_3digit_naics_all_years_df = temp_cbp_lq_3digit_naics_all_years_df[['county_fips','qp1', 'ap', 'est']].groupby(['county_fips']).sum().reset_index()
    
    
    # Pivot all the naics codes up into columns
    pivot_cbp_lq_2digit_naics_all_years_df = pd.pivot_table(temp_cbp_lq_3digit_naics_all_years_df, values='location_quotient_county_us', index=['county_fips'],
                        columns=['naics'], aggfunc=np.average, fill_value=0)


    # Add the aggregations to the pivot table
    merge_cbp_lq_2digit_naics_all_years_df = pivot_cbp_lq_2digit_naics_all_years_df.merge(group_cbp_lq_3digit_naics_all_years_df, on='county_fips', how='left')
    merge_cbp_lq_2digit_naics_all_years_df['year'] = year

    # Do I need to do this?
    merge_cbp_lq_2digit_naics_all_years_df.reset_index(inplace=True)
    all_data_cbp = pd.concat([all_data_cbp, merge_cbp_lq_2digit_naics_all_years_df])

FileTools.save_df_as_parquet(all_data_cbp, 'cbp_lq_3digit_naics_all_years_PIVOT.gzip')
FileTools.save_df_as_csv(all_data_cbp, 'cbp_lq_3digit_naics_all_years_PIVOT.csv')    

Working on year : 2010
Working on year : 2011
Working on year : 2012
Working on year : 2013
Working on year : 2014
Working on year : 2015
Working on year : 2016
Working on year : 2017
Working on year : 2018
Working on year : 2019
Working on year : 2020


'cbp_lq_3digit_naics_all_years_PIVOT.csv'

In [27]:
all_data_cbp.head()

,index,county_fips,113,114,115,211,212,213,221,236,...,712,713,721,722,811,812,813,qp1,ap,est
0,0,01001,5.555412,0.000000,0.000000,0.0,1.945867,0.0,3.227681,0.681620,...,0.00000,1.281011,1.475549,2.406618,0.915081,1.075647,2.585839,40430,174710,870
1,1,01003,0.000000,0.285288,0.000000,0.0,0.000000,0.0,0.000000,1.066502,...,0.57368,1.313262,1.722986,1.721987,0.971572,0.756357,1.509355,288364,1346818,4656
2,2,01005,31.637423,0.000000,6.209217,0.0,0.000000,0.0,0.000000,0.000000,...,0.00000,0.000000,0.590252,0.721443,0.509799,0.000000,0.655996,19058,83058,514
3,3,01007,50.045967,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,...,0.00000,0.000000,0.000000,0.822521,1.026759,1.284786,1.061995,9317,40640,292
4,4,01009,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,1.002452,...,0.00000,0.610964,0.000000,0.887612,2.018704,1.447656,1.658862,26070,113977,699


In [18]:
# all_data_cbp = FileTools.load_df_from_parquet('cbp_lq_3digit_naics_all_years_PIVOT.gzip')
print(all_data_cbp.shape)
all_data_cbp.head()
# DBTools.truncate_and_insert_df(all_data_cbp, "cbp_PIVOT")

(3125, 87)


naics,county_fips,113,114,115,211,212,213,221,236,237,...,623,624,711,712,713,721,722,811,812,813
0,01001,19.258521,0.000000,1.088295,0.0,5.927539,0.000000,3.084663,1.266255,1.296894,...,1.760669,0.818138,0.000000,0.000000,2.494317,1.178953,2.200564,0.931165,0.985815,2.506899
1,01003,0.434738,0.283913,0.000000,0.0,0.363784,0.019591,0.865132,1.519719,0.733203,...,0.878100,0.655100,0.200591,0.886143,1.782219,2.451151,1.839714,0.941701,0.746507,1.413468
2,01005,35.670628,0.000000,3.692662,0.0,0.000000,0.000000,2.658154,0.115809,0.000000,...,0.000000,0.082619,0.000000,0.000000,0.373166,0.551884,0.929455,0.663762,0.236436,0.853477
3,01007,17.005723,0.000000,4.849301,0.0,0.000000,0.000000,0.780958,0.133072,1.774191,...,0.000000,0.218351,0.000000,0.000000,0.000000,0.000000,0.773856,1.322033,0.257383,0.639068
4,01009,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,1.206961,1.420982,...,2.048118,0.692632,0.000000,0.000000,0.593446,0.000000,1.217843,1.839667,2.052264,2.033457


##

### Working on Economic Indicators

* CAINC4__ALL_AREAS_1969_2020.csv
* https://virginia.box.com/s/v70g9niz9att0381tapsnmtffksq7ahw

In [29]:
# Read in the data, there are some encoding issues so we need to explicitly specify the encoding.
_CAINC4__ALL_AREAS_1969_2020_df = pd.read_csv(FileTools.get_full_file_path('CAINC4__ALL_AREAS_1969_2020.csv'),low_memory=False, encoding = "ISO-8859-1" )

# Clean up the description field a bit.
_CAINC4__ALL_AREAS_1969_2020_df['Description'] = _CAINC4__ALL_AREAS_1969_2020_df.Description.str.strip().str.replace("/","")

# Seems like there is double data for 'Employer contributions for government social insurance' but they have different LineCodes so we can take out one of them.
# The other descriptions are in 30's and on so we will take out 62.
_CAINC4__ALL_AREAS_1969_2020_df = _CAINC4__ALL_AREAS_1969_2020_df[~(_CAINC4__ALL_AREAS_1969_2020_df.LineCode == 62)]
# DBTools.truncate_and_insert_df(_CAINC4__ALL_AREAS_1969_2020_df, "CAINC4__ALL_AREAS_1969_2020_df")

### Pivot the table so that county is row and all predictors are columns by year

In [32]:
all_data_ei = pd.DataFrame()
for i in range(1969,2021):  
# for i in range(1969,1970):
    year = str(i)
    print(f'Working on year : {year}')
    temp_CAINC4__ALL_AREAS_1969_2020_df = _CAINC4__ALL_AREAS_1969_2020_df[[year, 'GeoFIPS',  'Description']].copy()
    pivot_CAINC4__ALL_AREAS_1969_2020_df = pd.pivot_table(temp_CAINC4__ALL_AREAS_1969_2020_df, 
                                                    values=year
                                                    ,index=['GeoFIPS']
                                                    ,columns=['Description']
                                                    ,aggfunc=np.sum, fill_value=0)

    pivot_CAINC4__ALL_AREAS_1969_2020_df['year'] = year
    pivot_CAINC4__ALL_AREAS_1969_2020_df.reset_index(inplace=True)
    all_data_ei = pd.concat([all_data_ei, pivot_CAINC4__ALL_AREAS_1969_2020_df])   

FileTools.save_df_as_parquet(all_data_ei, 'CAINC4_ALL_AREAS_1969_2020_PIVOT.gzip')
FileTools.save_df_as_csv(all_data_ei, 'CAINC4_ALL_AREAS_1969_2020_PIVOT.csv')
# DBTools.truncate_and_insert_df(all_data_ei, "CAINC4__ALL_AREAS_1969_2020_PIVOT")

Working on year : 1969
Working on year : 1970
Working on year : 1971
Working on year : 1972
Working on year : 1973
Working on year : 1974
Working on year : 1975
Working on year : 1976
Working on year : 1977
Working on year : 1978
Working on year : 1979
Working on year : 1980
Working on year : 1981
Working on year : 1982
Working on year : 1983
Working on year : 1984
Working on year : 1985
Working on year : 1986
Working on year : 1987
Working on year : 1988
Working on year : 1989
Working on year : 1990
Working on year : 1991
Working on year : 1992
Working on year : 1993
Working on year : 1994
Working on year : 1995
Working on year : 1996
Working on year : 1997
Working on year : 1998
Working on year : 1999
Working on year : 2000
Working on year : 2001
Working on year : 2002
Working on year : 2003
Working on year : 2004
Working on year : 2005
Working on year : 2006
Working on year : 2007
Working on year : 2008
Working on year : 2009
Working on year : 2010
Working on year : 2011
Working on 

'CAINC4_ALL_AREAS_1969_2020_PIVOT.csv'

In [33]:
all_data_ei.head()

Description,GeoFIPS,Earnings by place of work,Employee and self-employed contributions for government social insurance,Employer contributions for employee pension and insurance funds 8,Employer contributions for government social insurance,Equals: Net earnings by place of residence,Farm income 2,Farm proprietors' income,Less: Contributions for government social insurance 5,Nonfarm personal income 1,...,"Plus: Dividends, interest, and rent 7",Plus: Personal current transfer receipts,Population (persons) 3,Proprietors employment,Proprietors' income 9,Supplements to wages and salaries,Total employment,Wage and salary employment,Wages and salaries,year
0,"""00000""",653665000,21204000,41314000,22587000,609767000,17275000,13407000,43791000,773954000,...,119140000,62322000,201298000,12331200,77578000,63901000,91053200,78722000,512186000,1969
1,"""01000""",8112293,278249,538937,272228,7692073,277960,230494,550477,9459755,...,1153840,891802,3440000,195784,953312,811165,1411234,1215450,6347816,1969
2,"""01001""",33797,1173,1463,990,56846,3544,2647,2163,66429,...,7626,5501,25166,1173,8895,2453,6630,5457,22449,1969
3,"""01003""",80410,3011,3358,2605,118705,6504,4501,5616,150687,...,23653,14833,56951,5122,16981,5963,20008,14886,57466,1969
4,"""01005""",36403,1318,1452,1142,38372,3977,3410,2460,47153,...,6010,6748,23818,2279,9442,2594,9428,7149,24367,1969
